In [2]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from prml.bayesnet.discrete import discrete

np.random.seed(1234)

b = discrete([0.1, 0.9])
f = discrete([0.1, 0.9])

g = discrete([[[0.9, 0.8], [0.8, 0.2]], [[0.1, 0.2], [0.2, 0.8]]], b, f)

print("b: ", b)
print("f: ", f)
print("g: ", g)

g.observe(0)

print("b: ", b)
print("f: ", f)
print("g: ", g)

b.observe(0)

print("b: ", b)
print("f: ", f)
print("g: ", g)

b:  DiscreteVariable(probability=[0.1 0.9])
f:  DiscreteVariable(probability=[0.1 0.9])
g:  DiscreteVariable(probability=[0.315 0.685])
b:  DiscreteVariable(probability=[0.1 0.9])
f:  DiscreteVariable(probability=[0.1 0.9])
g:  DiscreteVariable(observed=[1. 0.])
b:  DiscreteVariable(observed=[1. 0.])
f:  DiscreteVariable(probability=[0.1 0.9])
g:  DiscreteVariable(observed=[1. 0.])
